In [1]:
from lapis import lapis, noaa_dav, gps_time, flood_stats
import leafmap.leafmap as leafmap  # for ipyleaflet only
import dataretrieval.nwis as nwis

### Inputs: 
Coordinates (converted to circles), polygon features (shapefiles, gpkg, geojson etc., -- Only single part), or bounding boxes

In [2]:
locations_dict = [
    {
        'name': 'gage id 01650500',
        'data_type': 'coords',
        'value': (-77.0293611, 39.06552778), #longitude, latitude
        'buffer': 500, # in meters -- default value is 50 meters unless specified 
    },
    {
        'name': 'gage id 01651000',
        'data_type': 'coords',
        'value': (-76.96513889, 38.95255556), #longitude, latitude
        'buffer': 500, # in meters -- default value is 50 meters unless specified 
    },
    {
        'name': 'washington dc',
        'data_type': 'file',
        'value': 'demo_data/district_of_columbia_boundary.geojson'
    },
    {
        'name': 'augusta county, va',
        'data_type': 'bbox',
        'value': [-79.53330839559011, 37.88158664008918, -78.74939547673364, 38.477678774587105]
    }
]

### Find all lidar collections that intersect provided locations (using USIAEI)

USIAEI lists all lidar collections, but sometimes paths to EPT (Entwine Point Tiles) files are not included. Another source of these EPT files is NOAA's Digital Access Viewer (DAV). First step is to webscrape all of NOAA's collection names and corresponding EPT files if any and cross walk the records from USIAEI and NOAA.

In [3]:
find_lidar = lapis.SearchLidarInventory(locations_dict)
locations, collections = find_lidar.get_lidar_collections()

### Result(s)

In [4]:
locations[['name', 'geometry']]

,name,geometry
0,gage id01650500,"POLYGON ((-77.02364 39.06464, -77.02379 39.064..."
1,gage id 01651000,"POLYGON ((-76.95943 38.95167, -76.95957 38.951..."
2,washington dc,"POLYGON ((-77.00244 38.96567, -77.00150 38.964..."
3,"augusta county, va","POLYGON ((-79.53331 37.88159, -78.74940 37.881..."


In [5]:
collections.head(3)

,geometry,OBJECTID,ID,Title,DataType,Status,Links,pointspacing,verticalaccuracy,horizontalaccuracy,...,horizontaldatum,restrictions,leafOnOff,AlternateTitle,notes,RecordOwner,ContractSpec,Shape_Length,Shape_Area,name
0,"POLYGON ((-77.16494 39.30900, -77.16678 39.312...",161,28493,2018 MNCPPC Lidar: Montgomery and Prince Georg...,Lidar-Topo,Complete,"{""links"":[{""link"":""https:\/\/lidar.geodata.md....",0.7 m (spec),"{""vertAccuracy"":[{""VertDescription"":""4.6 cm RM...",0.5 m,...,NAD83 HARN,Public,None,None,Plans are to submit to USGS after acceptance.,USGS,USGS LBS 1.2,441920.168052,4.317304e+09,gage id01650500
1,"POLYGON ((-77.45889 39.21983, -77.47362 39.208...",876,24506,2008 Montgomery County Lidar,Lidar-Topo,Complete,"{""links"":[{""link"":""http:\/\/www6.montgomerycou...",Not Provided,"{""vertAccuracy"":[{""VertDescription"":""15 cm RMS...",1 m,...,NAD83,Other,None,None,,USGS,Unknown,221628.863882,2.181791e+09,gage id01650500
2,"POLYGON ((-77.11908 39.27012, -77.12978 39.271...",1556,42214,2021 MNCPPC Lidar: Montgomery and Prince Georg...,Lidar-Topo,Complete,None,0.35 m (spec),"{""vertAccuracy"":[{""VertDescription"":""10 cm (sp...",0.5 m,...,NAD83 HARN,Public,None,USGS: MD_2020MontgomeryPrinceGeorges_C22,None,USGS,Unknown,426673.478813,4.243600e+09,gage id01650500


# Map and explore lidar collections 

In [6]:
m = leafmap.Map(center=[39, -77], zoom=4)
m.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
)
# style = {
#     'color': "red",
#     'fillOpacity': 0.5}

m.add_gdf(locations, layer_name="locations", style={'color': 'red','fillColor': '#3366cc'})
# m.add_gdf(collections, layer_name="collections", style={'color': 'blue'})
m

Map(center=[39, -77], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

### Cross-reference NOAA DAV - by web scraping 

In [7]:
noaa_dav_gdf = noaa_dav.scrape_digital_coast_repo()

### Parse lidar collections and extract EPT (if any)

In [8]:
collections = lapis.ProcessLidarCollections(collections, locations, noaa_dav_gdf).execute()

Notable columns in this dataframe:
- `Title`: Lidar collection title
- `collectionyear`: from USIEI -- year of lidar collection
- `ept_usiaei`: EPT file path from USIAEI if present
- `noaa_id`: If a USIEI collection is present in NOAA DAV repo then the unique will be present
- `ID #`: same as `noaa_id` for cross-walk
- `ept_noaa`: EPT file paths pulled from NOAA DAV
- `ept`: reconciled EPT urls -- this cross-references USIAEI and NOAA DAV and used to retrieve point clouds
- `ept_crs`: CRS for EPT files which is needed for on-the-fly transformation
- `ept_gdf`: GeoDataframe of the original location feature reprojected to the native CRS of the EPT file

In [9]:
collections[['name', 'Title', 'collectionyear', 'collectiondate', 'meets3dep', 'Status']].head(3)

,name,Title,collectionyear,collectiondate,meets3dep,Status
0,gage id01650500,2018 MNCPPC Lidar: Montgomery and Prince Georg...,2018,"Feb 8 - Sep 27, 2018",Pending Contribution to 3DEP,Complete
1,gage id01650500,2008 Montgomery County Lidar,2008,"Feb 17, Mar 3, 5, 6, 11, 20, 24, 26, 29, Apr ...",No,Complete
2,gage id01650500,2021 MNCPPC Lidar: Montgomery and Prince Georg...,2021,2021,Pending Contribution to 3DEP,Complete


EPT files and/or lidar point clouds are stored in their native projection system. In the collections, geodataframe. The `ept_gdf` column stores the location geodataframe transformed to the native EPT CRS. 

## Enter the index number to extract `ept` url path and `ept_bbox` bounding box to retrieve the lidar point clouds using PDAL

In [10]:
index = 0
collection = collections.iloc[[index]]
collection[['Title', 'name']]

,Title,name
0,2018 MNCPPC Lidar: Montgomery and Prince Georg...,gage id01650500


In [12]:
count, arrays, metadata, demo_las = lapis.fetch_lidar(collection, geometry_method='polygon', write=True)

LAS write output enabled. Saving file under demo_data/demo_data/demo_lidar.las


# Visualizing the point cloud data

In [14]:
leafmap.view_lidar(demo_las, cmap='terrain', backend='pyvista')

Widget(value="<iframe src='http://localhost:65230/index.html?ui=P_0x2a00e143a90_0&reconnect=auto' style='width…

In [15]:
array = arrays[0] # 0 is for data and 1 is for datatype + column names

print('no.of points / count: ', count)
print('no. arrays: ', len(arrays))
# print('metadata: ', metadata)

no.of points / count:  4393761
no. arrays:  1


In [16]:
timestamps = gps_time.LidarTimestamps(arrays[0]).get_timestamps()

{'2018-02-28 14:22:15', '2018-02-28 14:22:17', '2018-02-28 14:07:00', '2018-02-28 14:07:30', '2018-02-28 12:01:30', '2018-02-28 14:07:09', '2018-02-28 14:22:20', '2018-02-28 14:06:59', '2018-02-28 14:22:10', '2018-02-28 14:22:11', '2018-02-28 12:01:39', '2018-02-28 12:01:36', '2018-02-28 14:06:58', '2018-02-28 14:22:14', '2018-02-28 14:07:21', '2018-02-28 14:22:26', '2018-02-28 12:01:35', '2018-02-28 14:07:11', '2018-02-28 14:22:12', '2018-02-28 14:07:25', '2018-02-28 14:06:55', '2018-02-28 14:07:29', '2018-02-28 14:07:26', '2018-02-28 14:07:23', '2018-02-28 14:07:22', '2018-02-28 14:22:16', '2018-02-28 14:22:18', '2018-02-28 14:07:02', '2018-02-28 12:01:38', '2018-02-28 14:22:25', '2018-02-28 14:07:31', '2018-02-28 12:01:42', '2018-02-28 14:07:06', '2018-02-28 14:07:08', '2018-02-28 14:07:18', '2018-02-28 14:22:13', '2018-02-28 14:22:19', '2018-02-28 14:07:04', '2018-02-28 12:01:33', '2018-02-28 14:22:22', '2018-02-28 14:06:56', '2018-02-28 12:01:31', '2018-02-28 14:07:20', '2018-02-2

In [24]:
sorted_timestamps = sorted(timestamps)

In [35]:
print(f"Total Duration of the Collection: {collection.collectiondate.values[0]} \nlidar collection start: {sorted_timestamps[0]} UTC & ends: {sorted_timestamps[-1]} UTC")

Total Duration of the Collection: Feb 8 - Sep 27, 2018 
lidar collection start: 2018-02-28 12:01:27 UTC & ends: 2018-02-28 14:22:28 UTC


### Pending workflow to extract rating curve at the time of the lidar collection

Substitue current rating curve created for colesville gage

In [2]:
site = "01650500"

What rating type to pick? exsa or base
`https://rconnect.usgs.gov/dataRetrieval/reference/readNWISrating.html`

In [5]:
rating_data = nwis.get_ratings(site=site, file_type="exsa") # or base?
rating_data = flood_stats.format_rating_data(rating_data)

columns renamed: {'INDEP': 'stage_ft', 'DEP': 'flow_cfs'}


In [46]:
data = nwis.get_ratings(site=site, file_type="base")

Pull peak flow stats from StreamStats API and convert it into a dataframe

In [13]:
peak_flow_stats = flood_stats.PeakFlowStatistics(site).process()

Columns rename: {'regressionType.code': 'pfs_aep_code', 'regressionType.name': 'pfs_aep_name', 'value': 'pfs_flow_cfs'}


In [11]:
peak_flow_stats[['pfs_aep_name', 'pfs_flow_cfs']]

,pfs_aep_name,pfs_flow_cfs
0,50-percent AEP flood,1254.0
1,20-percent AEP flood,2257.0
2,10-percent AEP flood,3236.0
3,4-percent AEP flood,4958.0
4,2-percent AEP flood,6688.0
5,1-percent AEP flood,8900.0
6,0.5-percent AEP flood,11720.0
7,0.2-percent AEP flood,16650.0
8,66.7-percent AEP flood,1016.0
9,80-percent AEP flood,790.0


Cross-reference AEP flow values with flow values from rating curve and find corresponding stage heights

In [16]:
peak_flow_stats = flood_stats.FlowToStage(
    peak_flow_stats, rating_curve
    ).find_stage_associated_with_aep_flow()

- `pfs_aep_name`: peak flow statistic (PFS) annual exceedance probability (AEP) name
- `pfs_aep_code`: PFS AEP alphanumeric code
---
- `pfs_flow_cfs`: PFS AEP flow/discharge in cubic feet per second
- `rc_flow_cfs`: rating curve (stage versus flow/discharge in cubic feet per second)
- `rc_stage_ft`: rating curve stage height in feet
---
- `pfs_flow_cms`: PFS AEP flow/discharge in cubic meters per second
- `rc_flow_cms`: rating curve (stage versus flow/discharge in cubic meters per second)
- `rc_stage_m`: rating curve stage height in meters

In [18]:
peak_flow_stats[['pfs_aep_name', 'pfs_flow_cms', 'rc_flow_cms', 'rc_stage_m']]

,pfs_aep_name,pfs_flow_cms,rc_flow_cms,rc_stage_m
0,50-percent AEP flood,35.509292,35.531662,2.267712
1,20-percent AEP flood,63.911063,64.033958,2.886456
2,10-percent AEP flood,91.633229,91.716198,3.084576
3,4-percent AEP flood,140.394793,140.172506,3.307080
4,2-percent AEP flood,189.382892,189.334753,3.563112
5,1-percent AEP flood,252.019697,251.940693,4.081272
6,0.5-percent AEP flood,331.873129,311.485019,4.843272
7,0.2-percent AEP flood,471.475051,311.485019,4.843272
8,66.7-percent AEP flood,28.769889,28.780933,2.039112
9,80-percent AEP flood,22.370288,22.371704,1.786128


to do list:
- add raster and hand grid
- visualize raster and hand grid